<a href="https://colab.research.google.com/github/GeraudBourdin/llm-scripts/blob/main/Galore_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade galore-torch
!pip install --upgrade accelerate
!pip install --upgrade bitsandbytes
!pip install --upgrade datasets
!pip install --upgrade transformers
!pip install --upgrade trl
!pip install --upgrade torch
!pip install --upgrade wandb

In [ ]:
import torch
import datasets
from transformers import TrainingArguments, AutoConfig, AutoTokenizer, AutoModelForCausalLM
import trl
import os
import wandb

%env WANDB_WATCH=all

wandb.login()

os.environ["WANDB_PROJECT"]="test_colab_galore"
# os.environ["WANDB_LOG_MODEL"] = "checkpoint"

train_dataset = datasets.load_dataset('imdb', split='train[:2%]')

args = TrainingArguments(
    output_dir="./test-galore",
    # max_steps=100,
    per_device_train_batch_size=2,

    report_to="wandb",
    run_name="test_colab_galore",
    logging_steps=10,

    optim="galore_adamw_layerwise",
    optim_args="rank=64, update_proj_gap=100, scale=0.10",
    optim_target_modules=[r".*attn.*", r".*mlp.*"],

    num_train_epochs=10,
    weight_decay=0.01,
    # lr_scheduler_type="cosine_with_restarts",
    lr_scheduler_type="cosine",
    learning_rate=1e-3,

)

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

config = AutoConfig.from_pretrained(model_id)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_config(config, torch_dtype=torch.bfloat16).to(0)

trainer = trl.SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    dataset_text_field='text',
    max_seq_length=512,
)

trainer.train()
wandb.finish()

wandb: Currently logged in as: gbourdin. Use `wandb login --relogin` to force relogin
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Activated GaLoRE fine-tuning, depending on your model size and hardware, the training might take a while before starting. Please be patient !
/usr/local/lib/python3.10/dist-packages/galore_torch/adamw.py:48: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,10.750800
20,9.035500
30,8.211400
40,7.925400
50,7.804800
60,7.633800
70,7.357800
80,7.444000
90,7.560100
100,7.419900
